In [1]:
from transformers import RobertaTokenizer, RobertaForMaskedLM

import os
import torch
import tqdm

tokenizer = RobertaTokenizer.from_pretrained('./pretrained')
model = RobertaForMaskedLM.from_pretrained('./pretrained', output_hidden_states=True)
model.to("cuda:0")

txt_root = "./data/text_c10"
img_root = "./data/CUB_200_2011/images"
max_len = 0
for bird in tqdm.tqdm(os.listdir(txt_root)):
    if ".txt" in bird:
        continue
    bird_root = os.path.join(txt_root, bird)
    for filename in os.listdir(bird_root):
        if ".txt" not in filename:
            continue
        with open(os.path.join(bird_root, filename)) as f:
            input_txt = f.readlines()[0]
        inputs = tokenizer(input_txt, return_tensors="pt").to("cuda:0")

        outputs = model(**inputs)
        saved_path = os.path.join(img_root, bird, filename) + ".pt"
        torch.save(outputs.hidden_states[-1], saved_path)
        if outputs.hidden_states[-1].shape[1] > max_len:
            max_len = outputs.hidden_states[-1].shape[1]

100%|██████████| 202/202 [01:35<00:00,  2.11it/s]


In [2]:
max_len

80